In [6]:
import pandas as pd
import datetime
import time
import sqlalchemy as db
import psycopg2

data = pd.read_csv('/Users/xiaokeai/Desktop/adv/trades.csv')

In [7]:
engine = db.create_engine('postgresql://localhost:5432/postgres')
connection = engine.connect()
metadata = db.MetaData()  

In [9]:
data.head(10)

,time_stamp,exchange,symbol,side,price,amount
0,1.637270e+09,BINANCE,BTC-USDT,sell,58162.31,0.00431
1,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.00367
2,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.00246
3,1.637270e+09,BINANCE,BTC-USDT,sell,58162.31,0.00026
4,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.00900
5,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.01000
6,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.00051
7,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.00599
8,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.04010
9,1.637270e+09,BINANCE,BTC-USDT,buy,58162.32,0.00621


In [17]:
file = "/Users/xiaokeai/Desktop/adv/trades.csv"
df = pd.read_csv(file)

#to standard timestamp format
df['time_stamp'] = pd.to_datetime(df['time_stamp'],unit='s')

df.insert(1, 'ts_to', df['time_stamp'].copy())
df.rename(columns={"time_stamp": "ts_from"}, inplace=True)
df['ts_to'] = df['ts_to'].shift(periods=-1)
df.dropna(inplace=True)
df

create_table = "CREATE TABLE trades(ts_from TIMESTAMP,ts_to TIMESTAMP,exchange VARCHAR,symbol VARCHAR,side VARCHAR,price float,amount float)"
connection.execute("DROP TABLE IF EXISTS trades")
connection.execute(create_table)
connection.execute("alter table trades drop column exchange;")


#load data
df.to_sql("trades", con=engine, if_exists="append", index=False)


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "exchange" of relation "trades" does not exist
LINE 1: INSERT INTO trades (ts_from, ts_to, exchange, symbol, side, ...
                                            ^

[SQL: INSERT INTO trades (ts_from, ts_to, exchange, symbol, side, price, amount) VALUES (%(ts_from)s, %(ts_to)s, %(exchange)s, %(symbol)s, %(side)s, %(price)s, %(amount)s)]
[parameters: ({'ts_from': datetime.datetime(2021, 11, 18, 21, 13, 17, 436999), 'ts_to': datetime.datetime(2021, 11, 18, 21, 13, 17, 472999), 'exchange': 'BINANCE', 'symbol': 'BTC-USDT', 'side': 'sell', 'price': 58162.31, 'amount': 0.00431}, {'ts_from': datetime.datetime(2021, 11, 18, 21, 13, 17, 472999), 'ts_to': datetime.datetime(2021, 11, 18, 21, 13, 17, 483000), 'exchange': 'BINANCE', 'symbol': 'BTC-USDT', 'side': 'buy', 'price': 58162.32, 'amount': 0.00367}, {'ts_from': datetime.datetime(2021, 11, 18, 21, 13, 17, 483000), 'ts_to': datetime.datetime(2021, 11, 18, 21, 13, 17, 532000), 'exchang

In [12]:
df

,ts_from,ts_to,exchange,symbol,side,price,amount
0,2021-11-18 21:13:17.436999936,2021-11-18 21:13:17.472999936,BINANCE,BTC-USDT,sell,58162.31,0.00431
1,2021-11-18 21:13:17.472999936,2021-11-18 21:13:17.483000064,BINANCE,BTC-USDT,buy,58162.32,0.00367
2,2021-11-18 21:13:17.483000064,2021-11-18 21:13:17.532000000,BINANCE,BTC-USDT,buy,58162.32,0.00246
3,2021-11-18 21:13:17.532000000,2021-11-18 21:13:17.655000064,BINANCE,BTC-USDT,sell,58162.31,0.00026
4,2021-11-18 21:13:17.655000064,2021-11-18 21:13:18.305999872,BINANCE,BTC-USDT,buy,58162.32,0.00900
...,...,...,...,...,...,...,...
250020,2021-11-19 00:57:38.509999872,2021-11-19 00:57:38.548999936,BINANCE,BTC-USDT,buy,57426.48,0.00611
250021,2021-11-19 00:57:38.548999936,2021-11-19 00:57:38.848999936,BINANCE,BTC-USDT,sell,57426.47,0.00407
250022,2021-11-19 00:57:38.848999936,2021-11-19 00:57:38.938999808,BINANCE,BTC-USDT,buy,57426.48,0.00589
250023,2021-11-19 00:57:38.938999808,2021-11-19 00:57:39.048999936,BINANCE,BTC-USDT,sell,57426.47,0.02231


In [16]:
df.to_csv("trades_interval.csv")

In [15]:
candle = "
with minmaxtable as 
(select date_trunc('hour', ts_from) h, min(price) minprice, max(price) maxprice,min(ts_from) mintimes,max(ts_from) maxtimes from trades group by h) 

select ts_from hours, avg(openprice) openprice,avg(closeprice) closeprice,avg(minprice) minprice,avg(maxprice) maxprice 
from 

(select distinct minmaxtable.h ts_from,openprice,closeprice,minprice,maxprice 
from 
(select minmaxtable.h ts_from, trades.price openprice 
from trades, minmaxtable 
where trades.ts_from=minmaxtable.mintimes) as openpricetable, 
(select minmaxtable.h ts_from, trades.price closeprice 
from trades, minmaxtable 
where trades.ts_from=minmaxtable.maxtimes) as closepricetable, 
minmaxtable 
where openpricetable.ts_from=closepricetable.ts_from and minmaxtable.h=openpricetable.ts_from 
order by ts_from) as result 

group by ts_from; 
"

connection.execute(candle)

SyntaxError: EOL while scanning string literal (<ipython-input-15-ea0a415c763b>, line 1)